In [1]:
import ipywidgets as widgets
import json
import os
from os import system
from os.path import join,basename
from IPython.display import display
try:
    import glob2
except ImportError:
    system('python -m pip install glob2')
    import glob2
import requests

In [2]:
# get isAbout URLs are used in this project along with 
# isAbout labels and 
query='PREFIX nidm: <http://purl.org/nidash/nidm#> \
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \
       select distinct ?isabout_url ?isabout_label where { \
        ?s nidm:isAbout ?isabout_url . \
        ?isabout_url rdfs:label ?isabout_label .}'
query_enc = requests.utils.quote(query)

# Set URL for ReproLake GraphDB resource for OpenNeuro NIDM files
url = "http://nidm.scicrunch.io:7200/repositories/OpenNeuro?query=" + query_enc
payload='-H=Accept%3Aapplication%2Fsparql-results%2Bjson'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}
# send post request to graphDB server
isabout_query_response = requests.request("POST", url, headers=headers, data=payload)

In [3]:
#loop through isAbout and set up dictionary for UI
isAbout_term_labels={}
for c in isabout_query_response.iter_lines():
    # get query return for this row
    isAbout_url = str(c.decode()).split(',')[0]
    isAbout_label = str(c.decode()).split(',')[1]
    # skip first row which has binding labels from sparql query
    if isAbout_url == 'isabout_url':
        continue
    # if we haven't stored any source variables from this dataset
    if (isAbout_url not in isAbout_term_labels.keys()):
        # set top-level data key to datset number
        isAbout_term_labels[str(isAbout_url)] = str(isAbout_label)
        #print("%s = %s" %(isAbout_url,isAbout_label))

In [4]:
# get image types for all OpenNeuro NIDM files
query = 'PREFIX nidm: <http://purl.org/nidash/nidm#> \
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \
        PREFIX prov: <http://www.w3.org/ns/prov#> \
        PREFIX dct: <http://purl.org/dc/terms/> \
        select distinct ?imageUsageType ?imageContrastType ?task where { \
            ?image_act a nidm:AcquisitionObject ; \
            nidm:hadAcquisitionModality nidm:MagneticResonanceImaging ; \
            nidm:hadImageContrastType ?imageContrastType ; \
            nidm:hadImageUsageType ?imageUsageType . \
        optional{?image_act nidm:Task ?task .} . }'


query_enc = requests.utils.quote(query)

# Set URL for ReproLake GraphDB resource for OpenNeuro NIDM files
url = "http://nidm.scicrunch.io:7200/repositories/OpenNeuro?query=" + query_enc
payload='-H=Accept%3Aapplication%2Fsparql-results%2Bjson'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}
# send post request to graphDB server
image_types_query_response = requests.request("POST", url, headers=headers, data=payload)

In [5]:
#loop through image types and set up dictionary for UI
image_types=[]
for c in image_types_query_response.iter_lines():
    # get query return for this row
    image_usage_type = (str(c.decode()).split(',')[0])
    image_contrast_type = (str(c.decode()).split(',')[1])
    # not used for now...
    # image_task = str(c.decode()).split(',')[2]
    # skip first row which has binding labels from sparql query
    if image_usage_type == 'imageUsageType':
        continue
    # if we haven't stored any source variables from this dataset
    if (image_usage_type not in image_types):
        image_types.append(image_usage_type)
    if (image_contrast_type not in image_types):
        image_types.append(image_contrast_type)

print("%s" %(image_types))

['http://purl.org/nidash/nidm#Anatomical', 'http://purl.org/nidash/nidm#T2Weighted', 'http://purl.org/nidash/nidm#T1Weighted', 'http://purl.org/nidash/nidm#Functional', 'http://purl.org/nidash/nidm#FlowWeighted', 'http://purl.org/nidash/nidm#DiffusionTensor', 'http://purl.org/nidash/nidm#DiffusionWeighted']


In [160]:
# temporary variables for query
global currentQueryTerm, queryTerms,currentImageQueryTerm
currentQueryTerm =''
queryTerms = []
imageTerms = []
# handler for drop down
def dropdown_eventhandler(change):
    #if change['name'] == 'value' and (change['new'] != change['old']):
    if change['name'] == 'value' and (change['new']):
        # set currentQueryTerm to selection
        #print("current query term changed %s" %change['new'])
        global currentQueryTerm
        currentQueryTerm = change['new']
        
# handler for drop down
def image_dropdown_eventhandler(change):
    if change['name'] == 'value' and (change['new']):
        # set currentImageQueryTerm to selection
        #print("current query term changed %s" %change['new'])
        global currentImageQueryTerm
        currentImageQueryTerm = change['new']
           
# add handlers for simple GUI
def btn_addConceptToQuery(obj):
    global currentQueryTerm
    global queryTerms
    if (currentQueryTerm not in queryTerms) and (currentQueryTerm != ''):
        #add to query Terms
        queryTerms.append(currentQueryTerm)
        print("Query: %s " %queryTerms)
def btn_addImageTypeToQuery(obj):
    global currentQueryTerm
    global imageTerms,queryTerms
    if (currentImageQueryTerm not in imageTerms) and (currentImageQueryTerm != ''):
        #add to query Terms
        imageTerms.append(currentImageQueryTerm)
        print("Query: %s %s " %(imageTerms,queryTerms))
# handlers for buttons
def btn_reset(obj):
    global queryTerms,imageTerms
    queryTerms = []
    imageTerms = []
    print("query terms reset: %s %s" %(queryTerms,imageTerms))

def ANDQuery(obj):
    global queryTerms,imageTerms
    
    
    query = '''PREFIX nidm: <http://purl.org/nidash/nidm#> 
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
                PREFIX prov: <http://www.w3.org/ns/prov#> 
                PREFIX dct: <http://purl.org/dc/terms/> 
                PREFIX dctypes: <http://purl.org/dc/dcmitype/> 
                PREFIX ndar: <https://ndar.nih.gov/api/datadictionary/v2/dataelement/> 
                
                select distinct ?url where { '''
    
    for term in queryTerms:
        # find isAbout url matching term
        isabout_url = [key for key,value in isAbout_term_labels.items() if value == term]
        query = query + '''
                    ?uuid nidm:isAbout <%s> . 
                    ?entity_uuid ?uuid ?value . 
                    ?entity_uuid prov:wasGeneratedBy ?activity_uuid . 
        ''' %(' '.join(map(str, isabout_url)))
    
    query = query + '''
                    ?activity_uuid prov:qualifiedAssociation ?bnode . 
                    ?bnode prov:agent ?agent_uuid . 
                    
                    ?agent_uuid ndar:src_subject_id ?ID . 
                    ?imaging_activity prov:qualifiedAssociation ?bnode2 . 
                    ?bnode2 prov:agent ?agent_uuid . 
                    
    ''' 
    
    for term in imageTerms:
    
        query = query + '''
            ?imaging_entity prov:wasGeneratedBy ?imaging_activity ; 
                        nidm:hadImageContrastType <%s> ; 
                        prov:wasGeneratedBy/dct:isPartOf/dct:isPartOf ?project . 
        
        ''' %(term)
                    
    query = query + '''
            ?project dctypes:title ?title ;
                            prov:Location ?url .}
    
    '''   


    #print(query)
    query_enc = requests.utils.quote(query)

    # Set URL for ReproLake GraphDB resource for OpenNeuro NIDM files
    url = "http://nidm.scicrunch.io:7200/repositories/OpenNeuro?query=" + query_enc
    payload='-H=Accept%3Aapplication%2Fsparql-results%2Bjson'
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded'
    }
    # send post request to graphDB server
    print("Running AND Query.... ")
    projects_query = requests.request("POST", url, headers=headers, data=payload)
    
    print("Done... ")
    and_query_results = []
    for c in projects_query.iter_lines():
        #line = c.decode('utf-8').split(",")
        and_query_results.append(c.decode('utf-8').replace("\n","").replace("\r",""))

    print("Matching Datasets")
    for result in and_query_results:
        if result == 'url':
            continue
        if "file://Volumes/Research/datasets.datalad.org/openneuro/" in result:
            print(result.replace("file://Volumes/Research/datasets.datalad.org/openneuro/",
                         "https://openneuro.org/datasets/"))
        else:
            print(result)
        

In [161]:
# bring up a new selector for adding a concept to the query
queryTermSelector = widgets.Dropdown(options=list(isAbout_term_labels.values()))
queryTermSelector.observe(dropdown_eventhandler)
queryImageSelector = widgets.Dropdown(options=image_types)
queryImageSelector.observe(image_dropdown_eventhandler)
# create a little user interface to query using isAbout concepts
addConceptToQuery = widgets.Button(description='Add Concept')
addConceptToQuery.on_click(btn_addConceptToQuery)
addImageToQuery = widgets.Button(description='Add Image Type')
addImageToQuery.on_click(btn_addImageTypeToQuery)
doANDQuery = widgets.Button(description='Run AND query')
doANDQuery.on_click(ANDQuery)
#doORQuery = widgets.Button(description='Run OR query')
#doORQuery.on_click(ORQuery)
resetQuery = widgets.Button(description='Reset query')
resetQuery.on_click(btn_reset)
print("Please select concepts from the dropdown to include in a query:")
display(queryTermSelector)
display(addConceptToQuery)
display(queryImageSelector)
display(addImageToQuery)
display(doANDQuery)
display(resetQuery)

Please select concepts from the dropdown to include in a query:


Dropdown(options=('age', 'sex', 'gender', 'handedness assessment', 'ethnicity', 'anxiety', 'generalized anxiet…

Button(description='Add Concept', style=ButtonStyle())

Dropdown(options=('http://purl.org/nidash/nidm#Anatomical', 'http://purl.org/nidash/nidm#T2Weighted', 'http://…

Button(description='Add Image Type', style=ButtonStyle())

Button(description='Run AND query', style=ButtonStyle())

Button(description='Reset query', style=ButtonStyle())